# SIFT features

In [1]:
import numpy as np
import cv2 as cv

img = cv.imread(r"C:\Users\student1\Desktop\Datasets\dress3_16_smaller\aim_15\input\marie_PL-EB5_dress3_16.JPG")
print(type(img))
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
 
sift = cv.SIFT_create()
kp = sift.detect(gray,None)
 
# print total number of keypoints detected
print("Number of keypoints Detected: ", len(kp))
img=cv.drawKeypoints(gray,kp,img)
cv.imshow('sift_keypoints.jpg',img)
 
cv.imwrite('sift_keypoints.jpg',img)

<class 'numpy.ndarray'>
Number of keypoints Detected:  7364


True

: 

# Read databse after sparse reconstruction

https://colmap.github.io/database.html

The database contains the following tables:

* cameras
* images
* keypoints
* descriptors
* matches
* two_view_geometries


# ! ! Better to use SQLite instead

In [1]:
import sqlite3

In [20]:
import colmap.scripts.python.database as coldb

import os
import argparse
import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument("--database_path", default="gerrard-hall-smaller/database.db")
args = parser.parse_args(args=[])

if os.path.exists(args.database_path):
    print("Database exists")
    # return

# Open the database.

conn = coldb.COLMAPDatabase.connect(args.database_path)

# Create a cursor object using the connection
db = conn.cursor()
# db.close()

Database exists


## Cameras

In [21]:
rows=db.execute("SELECT * FROM cameras")

for row in rows:
    camera_id, model, width, height, params, prior = row
    params = coldb.blob_to_array(params, np.float64)
    print("Camera ID", camera_id)
    print("Model", model)
    print("Width", width, "Height", height)
    print("Params", params)
    print("Prior", prior)
    print("#"*50)

Camera ID 1
Model 2
Width 5616 Height 3744
Params [3744. 2808. 1872.    0.]
Prior 1
##################################################


## Images

In [22]:
conn.close()

## pair_id_to_images


In [10]:
import colmap.scripts.python.database as coldb

img1, img2= coldb.pair_id_to_image_ids(8589934608)
print(img1, img2)

4.0 20


In [9]:
MAX_IMAGE_ID = 2 ** 31 - 1
print(MAX_IMAGE_ID, type(MAX_IMAGE_ID))

2147483647 <class 'int'>


# Read sparse reconstrunctions

* Convert .bin file to .txt file

In [2]:
import subprocess
import os

# Define the command as a list
cmd = [
    "python",
    "colmap/scripts/python/read_write_model.py",
    "--input_model", "gerrard-hall-smallest/sparse/0",
    "--input_format", ".bin",
    "--output_model", "gerrard-hall-smallest/sparse/0",
    "--output_format", ".txt",
]
while True:
    try:
        # Run the command
        subprocess.run(cmd) 
        break
    except FileNotFoundError as e:
        print(e)
        # use os to make directories


num_cameras: 1
num_images: 4
num_points3D: 1566


# Reading .ply file
and checking for invalid vertices in point cloud

In [32]:
from plyfile import PlyData, PlyElement
import numpy as np



with open('gerrard-hall-smaller/dense/0/sparse/test_export.ply', 'rb') as f:
    plydata = PlyData.read(f)

    for ply_element in plydata.elements:
        print(dir(ply_element))
        print(ply_element) 
        print(type(ply_element))
        print(ply_element.data.shape, ply_element.data.dtype,)
        print(ply_element.properties)
        print(ply_element.data[0])
        print("#"*50)

    x_min, y_min, z_min = np.inf, np.inf, np.inf
    x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

    for i in range(ply_element.count):
        # print(ply_element.data[i], type(ply_element.data[i]), type(ply_element.data[i][0]),)
    

        # if i > 10:
        #     break
        # check if x y z are inf or nan
        # if np.isinf(ply_element.data[i][0]) or np.isnan(ply_element.data[i][0]):
        #     print("x is inf or nan")
        # if np.isinf(ply_element.data[i][1]) or np.isnan(ply_element.data[i][1]):
        #     print("y is inf or nan")
        # if np.isinf(ply_element.data[i][2]) or np.isnan(ply_element.data[i][2]):
        #     print("z is inf or nan")
        if ply_element.data[i][0] < x_min:
            x_min = ply_element.data[i][0]
        if ply_element.data[i][1] < y_min:
            y_min = ply_element.data[i][1]
        if ply_element.data[i][2] < z_min:
            z_min = ply_element.data[i][2]

        if ply_element.data[i][0] > x_max:
            x_max = ply_element.data[i][0]
        if ply_element.data[i][1] > y_max:
            y_max = ply_element.data[i][1]
        if ply_element.data[i][2] > z_max:
            z_max = ply_element.data[i][2]
    print("#"*50)

    print("x_min", x_min)
    print("y_min", y_min)
    print("z_min", z_min)
    print("x_max", x_max)
    print("y_max", y_max)
    print("z_max", z_max)

['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_sanity', '_comments', '_count', '_data', '_get_comments', '_get_data', '_get_properties', '_have_list', '_index', '_name', '_properties', '_property_lookup', '_read', '_read_bin', '_read_mmap', '_read_txt', '_set_comments', '_set_data', '_set_properties', '_write', '_write_bin', '_write_txt', 'comments', 'count', 'data', 'describe', 'dtype', 'header', 'name', 'ply_property', 'properties']
element vertex 41752
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
<class 'plyfile.PlyElement'>
(41752,) [('x', '<f4'

# .bin file basics

In [13]:
data = [100,25,34]
buffer = bytes(data)
print(buffer)
f = open("binary.txt", "bw")
f.write(buffer)
f.close()

b'd\x19"'


In [14]:
with open("binary.txt", "br") as f:
    data = f.read()
    print(data)
    print(list(data))

b'd\x19"'
[100, 25, 34]


# COLMAP

how 3DGS ```convert.py``` uses it

In [2]:
from argparse import ArgumentParser 
import logging
from argparse import ArgumentParser
import shutil
import os


In [3]:

parser = ArgumentParser("Colmap converter")
#this name shows when writing --help
parser.add_argument("--no_gpu", action='store_true')
# The action keyword argument specifies how the command-line arguments should be handled
# here no_gpu will store True when --no_gpu is passed
parser.add_argument("--skip_matching", action='store_true')
# will store True when --skip_matching is passed, by default it is False
parser.add_argument("--source_path", "-s", required=True, type=str)
parser.add_argument("--camera", default="OPENCV", type=str)
parser.add_argument("--colmap_executable", default="", type=str)
parser.add_argument("--resize", action="store_true")
parser.add_argument("--magick_executable", default="", type=str)

_StoreAction(option_strings=['--magick_executable'], dest='magick_executable', nargs=None, const=None, default='', type=<class 'str'>, choices=None, help=None, metavar=None)

In [4]:

args = parser.parse_args(["--source_path", "output/gerrard-hall-smallest",
                        "--resize",
                        ])

for arg in vars(args):
    print(arg,":", getattr(args, arg))

colmap_command = '"{}"'.format(args.colmap_executable) if len(args.colmap_executable) > 0 else "colmap"
print(colmap_command)
magick_command = '"{}"'.format(args.magick_executable) if len(args.magick_executable) > 0 else "magick"
print(magick_command)
use_gpu = 1 if not args.no_gpu else 0

no_gpu : False
skip_matching : False
source_path : output/gerrard-hall-smallest
camera : OPENCV
colmap_executable : 
resize : True
magick_executable : 
colmap
magick


## Feature extraction

In [43]:
os.makedirs(args.source_path + "/distorted/sparse", exist_ok=True)
feat_extracton_cmd = colmap_command + " feature_extractor "\
        "--database_path " + args.source_path + "/distorted/database.db \
        --image_path " + args.source_path + "/input \
        --ImageReader.single_camera 1 \
        --ImageReader.camera_model " + args.camera + " \
        --SiftExtraction.use_gpu " + str(use_gpu)
print(feat_extracton_cmd)
# COLMAP CLI to extract SIFT features from images
# COLMAP feature_extractor
# --database_path: Path to the SQLite database file, will be created
# --image_path :NOTE name of images directory is input
# --ImageReader.single_camera 1: Ensures all images have the same camera model
# --ImageReader.camera_model: Camera model to use for all images, default is OPENCV model
# because default args.camera is OPENCV
# --SiftExtraction.use_gpu: Whether to use the GPU for SIFT extraction
exit_code = os.system(feat_extracton_cmd)
print(exit_code)

colmap feature_extractor --database_path output/gerrard-hall-smallest/distorted/database.db         --image_path output/gerrard-hall-smallest/input         --ImageReader.single_camera 1         --ImageReader.camera_model OPENCV         --SiftExtraction.use_gpu 1


I20240630 15:40:38.461405 139876361580544 misc.cc:198] 
Feature extraction
I20240630 15:40:39.165918 139875617570816 feature_extraction.cc:254] Processed file [1/4]
I20240630 15:40:39.165945 139875617570816 feature_extraction.cc:257]   Name:            IMG_2349.JPG
I20240630 15:40:39.165948 139875617570816 feature_extraction.cc:283]   Dimensions:      5806 x 3815
I20240630 15:40:39.165951 139875617570816 feature_extraction.cc:286]   Camera:          #1 - OPENCV
I20240630 15:40:39.165953 139875617570816 feature_extraction.cc:289]   Focal Length:    6967.20px
I20240630 15:40:39.165961 139875617570816 feature_extraction.cc:302]   Features:        12134
I20240630 15:40:39.237188 139875617570816 feature_extraction.cc:254] Processed file [2/4]
I20240630 15:40:39.237222 139875617570816 feature_extraction.cc:257]   Name:            IMG_2354.JPG
I20240630 15:40:39.237237 139875617570816 feature_extraction.cc:283]   Dimensions:      5806 x 3815
I20240630 15:40:39.237249 139875617570816 feature_e

0


I20240630 15:40:39.380965 139876361580544 timer.cc:91] Elapsed time: 0.015 [minutes]


## Feature matching

In [44]:
feat_matching_cmd = colmap_command + " exhaustive_matcher \
    --database_path " + args.source_path + "/distorted/database.db \
    --SiftMatching.use_gpu " + str(use_gpu)
# exhausitive matching is used to match features between images
# database path same as above
# SiftMatching.use_gpu: Use GPU is used in last step
# No change in source_path tree structure only database.db is updated
exit_code = os.system(feat_matching_cmd)
if exit_code != 0:
    logging.error(f"Feature matching failed with code {exit_code}. Exiting.")
    exit(exit_code)

I20240630 15:42:11.935719 140345719013376 misc.cc:198] 
Exhaustive feature matching
I20240630 15:42:12.002760 140345719013376 feature_matching.cc:231] Matching block [1/1, 1/1]
I20240630 15:42:12.192990 140345719013376 feature_matching.cc:46]  in 0.190s
I20240630 15:42:12.196214 140345719013376 timer.cc:91] Elapsed time: 0.004 [minutes]


## Bundle adjustment

In [45]:
mapper_cmd = (colmap_command + " mapper \
    --database_path " + args.source_path + "/distorted/database.db \
    --image_path "  + args.source_path + "/input \
    --output_path "  + args.source_path + "/distorted/sparse \
    --Mapper.ba_global_function_tolerance=0.000001")
# Mapper is the main command to run the bundle adjustment
# database, image paths same as above
# output path is where the output will be stored
# bundle adjustment global function tolerance
# source_path tree structure
# source_path
#   |---input
#   |---distorated
#       |---database.db
#       |---sparse
#           |---0
#           |---1 and so on. normally we need/get a single consolidated model
exit_code = os.system(mapper_cmd)
if exit_code != 0:
    logging.error(f"Mapper failed with code {exit_code}. Exiting.")
    exit(exit_code)

I20240630 15:43:21.412920 140301693018112 misc.cc:198] 
Loading database
I20240630 15:43:21.455691 140301693018112 database_cache.cc:54] Loading cameras...
I20240630 15:43:21.458876 140301693018112 database_cache.cc:64]  1 in 0.003s
I20240630 15:43:21.458942 140301693018112 database_cache.cc:72] Loading matches...
I20240630 15:43:21.463934 140301693018112 database_cache.cc:78]  5 in 0.005s
I20240630 15:43:21.463981 140301693018112 database_cache.cc:94] Loading images...
I20240630 15:43:21.496069 140301693018112 database_cache.cc:143]  4 in 0.032s (connected 4)
I20240630 15:43:21.496082 140301693018112 database_cache.cc:154] Building correspondence graph...
I20240630 15:43:21.511308 140301693018112 database_cache.cc:190]  in 0.015s (ignored 0)
I20240630 15:43:21.511322 140301693018112 timer.cc:91] Elapsed time: 0.001 [minutes]
I20240630 15:43:21.519335 140301693018112 misc.cc:198] 
Finding good initial image pair
I20240630 15:43:21.548774 140301693018112 misc.cc:198] 
Initializing with 

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.735188e+03    0.00e+00    4.27e+05   0.00e+00   0.00e+00  1.00e+04        0    9.75e-02    1.29e-01
   1  4.767628e+03    9.68e+02    2.88e+06   0.00e+00   9.16e-01  2.36e+04        1    1.05e-02    1.40e-01
   2  4.414890e+03    3.53e+02    5.69e+06   8.55e+01   4.72e-01  2.36e+04        1    3.43e-03    1.43e-01
   3  3.825414e+03    5.89e+02    3.22e+06   4.08e+01   7.69e-01  2.80e+04        1    3.43e-03    1.47e-01
   4  3.584912e+03    2.41e+02    3.28e+06   2.09e+01   4.57e-01  2.80e+04        1    3.57e-03    1.50e-01
   5  3.270250e+03    3.15e+02    2.71e+06   6.49e+00   5.42e-01  2.80e+04        1    3.42e-03    1.54e-01
   6  3.005841e+03    2.64e+02    2.30e+06   7.77e+00   5.12e-01  2.80e+04        1    3.42e-03    1.57e-01
   7  2.793599e+03    2.12e+02    1.96e+06   1.76e+01   4.59e-01  2.80e+04        1    3.36e-03    1.60e-01
   8  2.655043e+03    1.39e+

I20240630 15:43:22.074715 140301693018112 misc.cc:205] 
Bundle adjustment report
------------------------
I20240630 15:43:22.074730 140301693018112 bundle_adjustment.cc:942] 
    Residuals : 5688
   Parameters : 4277
   Iterations : 101
         Time : 0.42367 [s]
 Initial cost : 1.00414 [px]
   Final cost : 0.609193 [px]
  Termination : No convergence

I20240630 15:43:22.075322 140301693018112 incremental_mapper.cc:160] => Filtered observations: 10
I20240630 15:43:22.075324 140301693018112 incremental_mapper.cc:167] => Filtered images: 0
I20240630 15:43:22.172732 140301693018112 misc.cc:198] 
Registering image #3 (3)
I20240630 15:43:22.172744 140301693018112 incremental_mapper.cc:495] => Image sees 985 / 3512 points
I20240630 15:43:22.200758 140301693018112 misc.cc:205] 
Pose refinement report
----------------------
I20240630 15:43:22.200769 140301693018112 bundle_adjustment.cc:942] 
    Residuals : 1602
   Parameters : 6
   Iterations : 31
         Time : 0.0235131 [s]
 Initial cost 

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  8.981576e+02    0.00e+00    3.83e+05   0.00e+00   0.00e+00  1.00e+04        0    4.07e-03    5.93e-03
   1  8.016385e+02    9.65e+01    8.38e+04   0.00e+00   9.91e-01  3.00e+04        1    5.54e-03    1.15e-02
   2  7.848436e+02    1.68e+01    1.88e+05   3.71e+01   7.42e-01  3.38e+04        1    5.29e-03    1.68e-02
   3  7.707823e+02    1.41e+01    8.07e+04   2.23e+01   9.44e-01  1.01e+05        1    5.34e-03    2.21e-02
   4  7.627134e+02    8.07e+00    3.44e+05   3.90e+01   5.20e-01  1.01e+05        1    5.19e-03    2.73e-02
   5  7.482643e+02    1.44e+01    3.01e+05   3.51e+01   7.36e-01  1.13e+05        1    5.47e-03    3.28e-02
   6  7.372284e+02    1.10e+01    3.48e+05   3.77e+01   6.32e-01  1.15e+05        1    5.38e-03    3.82e-02
   7  7.250021e+02    1.22e+01    3.37e+05   3.72e+01   6.82e-01  1.21e+05        1    5.29e-03    4.35e-02
   8  7.139144e+02    1.11e+

I20240630 15:43:23.004145 140301693018112 misc.cc:205] 
Bundle adjustment report
------------------------
I20240630 15:43:23.004152 140301693018112 bundle_adjustment.cc:942] 
    Residuals : 8068
   Parameters : 4622
   Iterations : 99
         Time : 0.535078 [s]
 Initial cost : 0.333652 [px]
   Final cost : 0.250683 [px]
  Termination : Convergence

I20240630 15:43:23.004932 140301693018112 incremental_mapper.cc:175] => Completed observations: 18
I20240630 15:43:23.005042 140301693018112 incremental_mapper.cc:178] => Merged observations: 0
I20240630 15:43:23.005194 140301693018112 incremental_mapper.cc:160] => Filtered observations: 0
I20240630 15:43:23.005196 140301693018112 incremental_mapper.cc:119] => Changed observations: 0.004462
I20240630 15:43:23.005199 140301693018112 misc.cc:198] 
Global bundle adjustment
I20240630 15:43:23.083860 140301693018112 misc.cc:205] 
Bundle adjustment report
------------------------
I20240630 15:43:23.083865 140301693018112 bundle_adjustment.cc:94

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.437797e+02    0.00e+00    1.71e+04   0.00e+00   0.00e+00  1.00e+04        0    4.09e-03    5.86e-03
   1  5.199366e+02    2.38e+01    3.77e+01   0.00e+00   1.00e+00  3.00e+04        1    5.54e-03    1.14e-02
   2  5.199195e+02    1.71e-02    1.50e+02   4.21e-01   9.98e-01  9.00e+04        1    5.23e-03    1.66e-02
   3  5.199164e+02    3.10e-03    6.98e+01   2.58e-01   9.96e-01  2.70e+05        1    5.57e-03    2.22e-02
   4  5.199160e+02    3.76e-04    3.12e+01   2.10e-01   9.97e-01  8.10e+05        1    5.29e-03    2.75e-02
   5  5.199153e+02    6.85e-04    1.92e+02   5.62e-01   9.98e-01  2.43e+06        1    5.50e-03    3.30e-02
   6  5.199138e+02    1.54e-03    1.26e+03   1.44e+00   9.70e-01  7.29e+06        1    5.23e-03    3.82e-02
   7  5.199120e+02    1.76e-03    5.03e+03   2.87e+00   7.12e-01  7.89e+06        1    5.34e-03    4.36e-02
   8  5.199104e+02    1.64e-

I20240630 15:43:23.245065 140301693018112 misc.cc:198] 
Registering image #4 (4)
I20240630 15:43:23.245075 140301693018112 incremental_mapper.cc:495] => Image sees 95 / 376 points
I20240630 15:43:23.248556 140301693018112 misc.cc:205] 
Pose refinement report
----------------------
I20240630 15:43:23.248567 140301693018112 bundle_adjustment.cc:942] 
    Residuals : 156
   Parameters : 6
   Iterations : 13
         Time : 0.000901937 [s]
 Initial cost : 0.835589 [px]
   Final cost : 0.819426 [px]
  Termination : Convergence

I20240630 15:43:23.248597 140301693018112 incremental_mapper.cc:40] => Continued observations: 78
I20240630 15:43:23.248755 140301693018112 incremental_mapper.cc:43] => Added observations: 17
I20240630 15:43:23.267418 140301693018112 misc.cc:205] 
Bundle adjustment report
------------------------
I20240630 15:43:23.267423 140301693018112 bundle_adjustment.cc:942] 
    Residuals : 2796
   Parameters : 479
   Iterations : 11
         Time : 0.0176539 [s]
 Initial cost 

   3  5.664226e+02    1.89e+00    3.13e+04   4.97e+00   9.73e-01  2.70e+05        1    5.35e-03    2.29e-02
   4  5.651471e+02    1.28e+00    1.55e+05   1.39e+01   6.95e-01  2.87e+05        1    5.56e-03    2.85e-02
   5  5.637227e+02    1.42e+00    1.10e+05   1.29e+01   8.42e-01  4.22e+05        1    5.72e-03    3.42e-02
   6  5.628360e+02    8.87e-01    1.46e+05   1.52e+01   6.58e-01  4.36e+05        1    5.47e-03    3.97e-02
   7  5.618814e+02    9.55e-01    9.78e+04   1.24e+01   8.23e-01  5.98e+05        1    5.52e-03    4.52e-02
   8  5.613455e+02    5.36e-01    1.02e+05   1.26e+01   7.09e-01  6.45e+05        1    5.53e-03    5.07e-02
   9  5.608909e+02    4.55e-01    6.38e+04   9.92e+00   8.41e-01  9.44e+05        1    5.55e-03    5.63e-02
  10  5.606618e+02    2.29e-01    5.93e+04   9.54e+00   7.56e-01  1.09e+06        1    5.58e-03    6.19e-02
  11  5.605071e+02    1.55e-01    3.13e+04   6.91e+00   8.83e-01  1.98e+06        1    5.42e-03    6.73e-02
  12  5.604475e+02    5.96e-

I20240630 15:43:23.504575 140301693018112 timer.cc:91] Elapsed time: 0.035 [minutes]


## Image undistortion

In [49]:
img_undist_cmd = (colmap_command + " image_undistorter \
    --image_path " + args.source_path + "/input \
    --input_path " + args.source_path + "/distorted/sparse/0 \
    --output_path " + args.source_path + "\
    --output_type COLMAP")
# image_path same as above. image folder name should be /input !!
# input_path to the sparse point cloud model
# new folder created for undistorated images in source_path/images
# output_type COLMAP ???
# now the source_path tree is
# source_path
#   |---input
#   |---distorated
#       |---database.db
#       |---sparse
#           |---0
#           |---1 and so on. normally we need/get a single consolidated model
#   |---images
exit_code = os.system(img_undist_cmd)
if exit_code != 0:
    logging.error(f"Mapper failed with code {exit_code}. Exiting.")
    exit(exit_code)

I20240630 15:49:15.093394 140512756174848 misc.cc:198] 
Reading reconstruction
I20240630 15:49:15.102104 140512756174848 image.cc:342] => Reconstruction with 4 images and 1535 points
I20240630 15:49:15.114693 140509474521088 misc.cc:198] 
Image undistortion
I20240630 15:49:15.120832 140509474521088 undistortion.cc:210] Undistorting image [1/4]
I20240630 15:49:17.161541 140509474521088 undistortion.cc:210] Undistorting image [2/4]
I20240630 15:49:17.161567 140509474521088 undistortion.cc:210] Undistorting image [3/4]
I20240630 15:49:17.161568 140509474521088 undistortion.cc:210] Undistorting image [4/4]
I20240630 15:49:17.161569 140509474521088 undistortion.cc:223] Writing reconstruction...
I20240630 15:49:17.183969 140509474521088 undistortion.cc:228] Writing configuration...
I20240630 15:49:17.186726 140509474521088 undistortion.cc:232] Writing scripts...
I20240630 15:49:17.189157 140509474521088 timer.cc:91] Elapsed time: 0.035 [minutes]


## Move files from sparse to sparse/0

In [50]:
files = os.listdir(args.source_path + "/sparse")
os.makedirs(args.source_path + "/sparse/0", exist_ok=True)
# Copy each file from the source directory to the destination directory
for file in files:
    if file == '0':
        continue
    source_file = os.path.join(args.source_path, "sparse", file)
    destination_file = os.path.join(args.source_path, "sparse", "0", file)
    shutil.move(source_file, destination_file)

## Resize images

In [6]:
if(args.resize):
    # Only if resize is set True
    print("Copying and resizing...")

    # Resize images.
    os.makedirs(args.source_path + "/images_2", exist_ok=True)
    os.makedirs(args.source_path + "/images_4", exist_ok=True)
    os.makedirs(args.source_path + "/images_8", exist_ok=True)
    # Get the list of files in the source directory
    files = os.listdir(args.source_path + "/images")
    # Shouldn't this be input instead of images???
    # Copy each file from the source directory to the destination directory
    for file in files:
        source_file = os.path.join(args.source_path, "images", file)

        destination_file = os.path.join(args.source_path, "images_2", file)
        shutil.copy2(source_file, destination_file)
        exit_code = os.system(magick_command + " mogrify -resize 50% " + destination_file)
        if exit_code != 0:
            logging.error(f"50% resize failed with code {exit_code}. Exiting.")
            exit(exit_code)

        destination_file = os.path.join(args.source_path, "images_4", file)
        shutil.copy2(source_file, destination_file)
        exit_code = os.system(magick_command + " mogrify -resize 25% " + destination_file)
        if exit_code != 0:
            logging.error(f"25% resize failed with code {exit_code}. Exiting.")
            exit(exit_code)

        destination_file = os.path.join(args.source_path, "images_8", file)
        shutil.copy2(source_file, destination_file)
        exit_code = os.system(magick_command + " mogrify -resize 12.5% " + destination_file)
        if exit_code != 0:
            logging.error(f"12.5% resize failed with code {exit_code}. Exiting.")
            exit(exit_code)

Copying and resizing...
